In [ ]:
!pip install osmnx networkx folium geopy


In [ ]:
import osmnx as ox
import networkx as nx
import folium
from geopy.geocoders import Nominatim


In [ ]:
# Define the place
place_name = "Pune, India"

# Fetch the road network of Pune
graph = ox.graph_from_place(place_name, network_type='drive')

# Get the center coordinates of Pune
geolocator = Nominatim(user_agent="geoapiExercises")
location = geolocator.geocode(place_name)

# Create a base map
pune_map = folium.Map(location=[location.latitude, location.longitude], zoom_start=12)

# Display the map
pune_map


GeocoderInsufficientPrivileges: Non-successful status code 403

In [ ]:
# Query for hospitals in Pune from OpenStreetMap using the updated method
tags = {'amenity': 'hospital'}
hospitals = ox.features_from_place(place_name, tags)

# Plot hospitals on the map
for idx, hospital in hospitals.iterrows():
    # If the geometry is a polygon, take the centroid
    if hospital.geometry.geom_type == 'Polygon':
        lat = hospital.geometry.centroid.y
        lon = hospital.geometry.centroid.x
    # If the geometry is a point, use its coordinates
    elif hospital.geometry.geom_type == 'Point':
        lat = hospital.geometry.y
        lon = hospital.geometry.x
    else:
        continue  # Skip other types of geometries

    # Add the hospital marker to the map
    folium.Marker([lat, lon], 
                  popup=hospital.get('name', 'Unnamed hospital'), 
                  icon=folium.Icon(color='red', icon='info-sign')).add_to(pune_map)

# Display the map with hospitals
pune_map


In [ ]:
geolocator = Nominatim(user_agent="my_pune_map_project")


In [ ]:
location = {"lat": 18.5204, "lon": 73.8567}  # Latitude and Longitude of Pune


In [ ]:
import osmnx as ox

# Query for hospitals in Pune from OpenStreetMap
place_name = "Pune, India"
tags = {'amenity': 'hospital'}
hospitals = ox.features_from_place(place_name, tags)

# Check if the data has the 'emergency' column, if not, add a placeholder
if 'emergency' not in hospitals.columns:
    hospitals['emergency'] = None  # Add 'None' for hospitals without emergency data


In [ ]:
# Save hospital data to a CSV file, including emergency info
hospitals[['name', 'amenity', 'emergency', 'geometry']].to_csv('pune_hospitals.csv', index=False)



In [ ]:
import pandas as pd
from shapely import wkt

# Load the CSV file
hospital_data = pd.read_csv('pune_hospitals.csv')

# Filter hospitals with emergency services
emergency_hospitals = hospital_data[hospital_data['emergency'] == 'yes'].copy()  # Use .copy() to avoid the warning

# Convert the 'geometry' column back to coordinates
emergency_hospitals.loc[:, 'geometry'] = emergency_hospitals['geometry'].apply(wkt.loads)

# Extract latitude and longitude from geometry
emergency_hospitals.loc[:, 'lat'] = emergency_hospitals['geometry'].apply(lambda x: x.centroid.y if x.geom_type == 'Polygon' else x.y)
emergency_hospitals.loc[:, 'lon'] = emergency_hospitals['geometry'].apply(lambda x: x.centroid.x if x.geom_type == 'Polygon' else x.x)


In [ ]:
import osmnx as ox

# Define the place
place_name = "Pune, India"

# Fetch the road network for driving
graph = ox.graph_from_place(place_name, network_type='drive')

# Convert the graph to a GeoDataFrame
nodes, edges = ox.graph_to_gdfs(graph)

# Filter only national highways from the 'highway' column
# 'highway' may contain a list of categories, including 'motorway' and 'trunk' for national highways
highways = edges[edges['highway'].isin(['motorway', 'trunk'])]

# Save the road network to a CSV file
highways[['name', 'highway', 'geometry']].to_csv('pune_highways.csv', index=False)


In [ ]:
import osmnx as ox
import pandas as pd
from shapely import wkt
import folium

# Define the place
place_name = "Pune, India"

# Fetch the road network for driving
graph = ox.graph_from_place(place_name, network_type='drive')

# Convert the graph to a GeoDataFrame
nodes, edges = ox.graph_to_gdfs(graph)

# Filter only national highways and primary roads
filtered_roads = edges[edges['highway'].isin(['motorway', 'trunk', 'primary'])]

# Load hospital data
hospital_data = pd.read_csv('pune_hospitals.csv')

# Filter for hospitals with emergency services
emergency_hospitals = hospital_data[hospital_data['emergency'] == 'yes'].copy()

# Convert 'geometry' column back to shapely objects
emergency_hospitals['geometry'] = emergency_hospitals['geometry'].apply(wkt.loads)

# Extract latitude and longitude from geometry
emergency_hospitals['lat'] = emergency_hospitals['geometry'].apply(lambda x: x.centroid.y if x.geom_type == 'Polygon' else x.y)
emergency_hospitals['lon'] = emergency_hospitals['geometry'].apply(lambda x: x.centroid.x if x.geom_type == 'Polygon' else x.x)

# Create a folium map centered on Pune
combined_map = folium.Map(location=[emergency_hospitals['lat'].mean(), emergency_hospitals['lon'].mean()], zoom_start=12)

# Plot national highways (motorway, trunk) and primary roads
for idx, road in filtered_roads.iterrows():
    coords = list(road['geometry'].coords)
    road_color = 'blue' if road['highway'] in ['motorway', 'trunk'] else 'blue'  # Orange for national highways, blue for primary roads
    
    # Check if 'name' is an array or list-like and handle appropriately
    road_name = road['name']
    if isinstance(road_name, (list, pd.Series)):
        road_name = road_name[0]  # Take the first name if multiple are present
    
    folium.PolyLine(locations=[(lat, lon) for lon, lat in coords], 
                    color=road_color, 
                    weight=5, 
                    opacity=0.5, 
                    popup=road_name if pd.notnull(road_name) else 'Unnamed Road').add_to(combined_map)

# Plot emergency hospitals on the map
for idx, hospital in emergency_hospitals.iterrows():
    folium.Marker([hospital['lat'], hospital['lon']], 
                  popup=hospital['name'], 
                  icon=folium.Icon(color='red', icon='info-sign')).add_to(combined_map)

# Display the combined map
combined_map


In [ ]:



import heapq
import osmnx as ox
from geopy.distance import geodesic
import folium
from folium.plugins import MousePosition

# Initialize map and add mouse position plugin to capture coordinates
def create_map(emergency_hospitals, graph):
    map_pune = folium.Map(location=[emergency_hospitals['lat'].mean(), emergency_hospitals['lon'].mean()], zoom_start=12)
    
    # Adding national highways and primary roads as before
    for idx, road in filtered_roads.iterrows():
        coords = list(road['geometry'].coords)
        road_color = 'blue' if road['highway'] in ['motorway', 'trunk'] else 'blue'
        
        # Handle the case where road['name'] might be a list or NaN
        road_name = road['name']
        if isinstance(road_name, list):
            road_name = road_name[0] if road_name else 'Unnamed Road'  # Take the first name if it's a list
        elif pd.isna(road_name):
            road_name = 'Unnamed Road'

        folium.PolyLine(locations=[(lat, lon) for lon, lat in coords], 
                        color=road_color, weight=5, opacity=0.5, popup=road_name).add_to(map_pune)

    # Adding hospitals to the map
    for idx, hospital in emergency_hospitals.iterrows():
        folium.Marker([hospital['lat'], hospital['lon']], 
                      popup=hospital['name'], 
                      icon=folium.Icon(color='red', icon='info-sign')).add_to(map_pune)

    # Adding mouse position plugin to capture coordinates
    mouse_position = MousePosition(position='topright')
    mouse_position.add_to(map_pune)

    return map_pune

# Define heuristic function using Euclidean distance
def heuristic(node, goal):
    return geodesic((node[1], node[0]), (goal[1], goal[0])).meters

# A* algorithm using the road network (Manhattan distance)
def a_star(graph, start_node, goal_node):
    pq = []  # Priority queue
    heapq.heappush(pq, (0, start_node))  # (cost, node)
    came_from = {start_node: None}
    cost_so_far = {start_node: 0}
    
    while pq:
        current_cost, current_node = heapq.heappop(pq)
        
        if current_node == goal_node:
            break
        
        # Replace 'ox.neighbors' with 'graph.neighbors' (since graph is a networkx graph)
        for neighbor in graph.neighbors(current_node):
            road_length = graph.edges[current_node, neighbor, 0]['length']
            new_cost = cost_so_far[current_node] + road_length
            
            if neighbor not in cost_so_far or new_cost < cost_so_far[neighbor]:
                cost_so_far[neighbor] = new_cost
                priority = new_cost + heuristic((graph.nodes[neighbor]['x'], graph.nodes[neighbor]['y']), 
                                                (graph.nodes[goal_node]['x'], graph.nodes[goal_node]['y']))
                heapq.heappush(pq, (priority, neighbor))
                came_from[neighbor] = current_node
                
    # Reconstruct path
    path = []
    current = goal_node
    while current != start_node:
        path.append(current)
        current = came_from[current]
    path.append(start_node)
    path.reverse()
    
    return path

# Function to add shortest path to the map
def plot_shortest_path(map_object, graph, path):
    route_coords = [(graph.nodes[node]['y'], graph.nodes[node]['x']) for node in path]
    folium.PolyLine(route_coords, color='green', weight=5, opacity=0.8).add_to(map_object)

# Main execution
map_pune = create_map(emergency_hospitals, graph)

# Capture user click and find the closest emergency hospital
user_click_location = (18.5204, 73.8567)  # Example coordinates, replace with user click input

# Find the nearest emergency hospital using Euclidean distance
emergency_hospitals['distance'] = emergency_hospitals.apply(
    lambda row: geodesic(user_click_location, (row['lat'], row['lon'])).meters, axis=1
)

# Get the nearest hospital
nearest_hospital = emergency_hospitals.loc[emergency_hospitals['distance'].idxmin()]
nearest_hospital_location = (nearest_hospital['lat'], nearest_hospital['lon'])

# Get the nearest nodes from the graph for both start and hospital using the updated function
start_node = ox.distance.nearest_nodes(graph, user_click_location[1], user_click_location[0])
goal_node = ox.distance.nearest_nodes(graph, nearest_hospital_location[1], nearest_hospital_location[0])

# Run A* algorithm to find the shortest path
shortest_path = a_star(graph, start_node, goal_node)

# Plot the shortest path on the map
plot_shortest_path(map_pune, graph, shortest_path)

# Display the map with the shortest path
map_pune


In [ ]:
import heapq
import osmnx as ox
from geopy.distance import geodesic
import folium

# Initialize map and add hospitals and roads
def create_map(emergency_hospitals, graph):
    map_pune = folium.Map(location=[emergency_hospitals['lat'].mean(), emergency_hospitals['lon'].mean()], zoom_start=12)
    
    # Adding national highways and primary roads as before
    for idx, road in filtered_roads.iterrows():
        coords = list(road['geometry'].coords)
        road_color = 'blue' if road['highway'] in ['motorway', 'trunk'] else 'blue'
        
        road_name = road['name']
        if isinstance(road_name, list):
            road_name = road_name[0] if road_name else 'Unnamed Road'
        elif pd.isna(road_name):
            road_name = 'Unnamed Road'

        folium.PolyLine(locations=[(lat, lon) for lon, lat in coords], 
                        color=road_color, weight=5, opacity=0.5, popup=road_name).add_to(map_pune)

    # Adding hospitals to the map
    for idx, hospital in emergency_hospitals.iterrows():
        folium.Marker([hospital['lat'], hospital['lon']], 
                      popup=hospital['name'], 
                      icon=folium.Icon(color='red', icon='info-sign')).add_to(map_pune)

    return map_pune

# Define heuristic function using Euclidean distance
def heuristic(node, goal):
    return geodesic((node[1], node[0]), (goal[1], goal[0])).meters

# A* algorithm using the road network (Manhattan distance)
def a_star(graph, start_node, goal_node):
    pq = []  # Priority queue
    heapq.heappush(pq, (0, start_node))  # (cost, node)
    came_from = {start_node: None}
    cost_so_far = {start_node: 0}
    
    while pq:
        current_cost, current_node = heapq.heappop(pq)
        
        if current_node == goal_node:
            break
        
        for neighbor in graph.neighbors(current_node):
            road_length = graph.edges[current_node, neighbor, 0]['length']
            new_cost = cost_so_far[current_node] + road_length
            
            if neighbor not in cost_so_far or new_cost < cost_so_far[neighbor]:
                cost_so_far[neighbor] = new_cost
                priority = new_cost + heuristic((graph.nodes[neighbor]['x'], graph.nodes[neighbor]['y']), 
                                                (graph.nodes[goal_node]['x'], graph.nodes[goal_node]['y']))
                heapq.heappush(pq, (priority, neighbor))
                came_from[neighbor] = current_node
                
    # Reconstruct path
    path = []
    current = goal_node
    while current != start_node:
        path.append(current)
        current = came_from[current]
    path.append(start_node)
    path.reverse()
    
    return path

# Function to add shortest path to the map
def plot_shortest_path(map_object, graph, path):
    route_coords = [(graph.nodes[node]['y'], graph.nodes[node]['x']) for node in path]
    folium.PolyLine(route_coords, color='green', weight=5, opacity=0.8).add_to(map_object)

# Function to get user location input
def get_user_location():
    locations = {
        'Shivajinagar': (18.5300, 73.8476),
        'Hadapsar': (18.5089, 73.9259),
        'Kothrud': (18.5074, 73.8077),
        'Aundh': (18.5590, 73.8078),
        'Viman Nagar': (18.5679, 73.9143)
    }

    print("Select a location in Pune:")
    for i, loc in enumerate(locations.keys(), start=1):
        print(f"{i}. {loc}")
    
    choice = int(input("Enter the number corresponding to your location: "))
    
    selected_location = list(locations.values())[choice - 1]
    print(f"Selected location: {list(locations.keys())[choice - 1]}")
    
    return selected_location

# Main execution
map_pune = create_map(emergency_hospitals, graph)

# Get user-selected location
user_click_location = get_user_location()

# Find the nearest emergency hospital using Euclidean distance
emergency_hospitals['distance'] = emergency_hospitals.apply(
    lambda row: geodesic(user_click_location, (row['lat'], row['lon'])).meters, axis=1
)

# Get the nearest hospital
nearest_hospital = emergency_hospitals.loc[emergency_hospitals['distance'].idxmin()]
nearest_hospital_location = (nearest_hospital['lat'], nearest_hospital['lon'])

# Get the nearest nodes from the graph for both start and hospital using the updated function
start_node = ox.distance.nearest_nodes(graph, user_click_location[1], user_click_location[0])
goal_node = ox.distance.nearest_nodes(graph, nearest_hospital_location[1], nearest_hospital_location[0])

# Run A* algorithm to find the shortest path
shortest_path = a_star(graph, start_node, goal_node)

# Plot the shortest path on the map
plot_shortest_path(map_pune, graph, shortest_path)

# Display the map with the shortest path
map_pune


In [ ]:
import heapq
import osmnx as ox
from geopy.distance import geodesic
import folium
from folium.plugins import MousePosition

# Predefined user locations
def get_user_location():
    locations = {
        'Shivajinagar': (18.5300, 73.8476),
        'Hadapsar': (18.5089, 73.9259),
        'Kothrud': (18.5074, 73.8077),
        'Aundh': (18.5590, 73.8078),
        'Viman Nagar': (18.5679, 73.9143),
        'Baner': (18.5603, 73.7890),
        'Wakad': (18.5971, 73.7707),
        'Magarpatta': (18.5139, 73.9317),
        'Pimpri': (18.6298, 73.7997),
        'Chinchwad': (18.6446, 73.7639),
        'Nigdi': (18.6510, 73.7629),
        'Bhosari': (18.6193, 73.8440),
        'Swargate': (18.5018, 73.8580),
        'Karve Nagar': (18.4861, 73.8279),
        'Camp': (18.5125, 73.8805),
        'Pashan': (18.5400, 73.7884),
        'Kharadi': (18.5510, 73.9346),
        'Dhanori': (18.5934, 73.8999),
        'Koregaon Park': (18.5362, 73.8925),
        'Mundhwa': (18.5298, 73.9260),
        'Warje': (18.4844, 73.8070),
        'Parvati': (18.4901, 73.8553),
        'Kondhwa': (18.4575, 73.8911),
        'Undri': (18.4536, 73.9052),
        'NIBM': (18.4655, 73.9028),
        'Bavdhan': (18.5159, 73.7731),
        'Balewadi': (18.5815, 73.7703),
        'Katraj': (18.4575, 73.8670),
        'Deccan': (18.5182, 73.8415),
        'Pimple Saudagar': (18.5932, 73.8074),
        'Akurdi': (18.6478, 73.7674),
        'Yerwada': (18.5675, 73.9024),
        'Sassoon Road': (18.5293, 73.8767),
        'Wagholi': (18.5671, 73.9803),
        'Sinhagad Road': (18.4735, 73.8355),
        'Dhankawadi': (18.4638, 73.8614),
        'Wanowrie': (18.4937, 73.8934),
        'Lohegaon': (18.6100, 73.9193),
        'Vadgaon Sheri': (18.5562, 73.9076),
        'Bibwewadi': (18.4711, 73.8714),
        'Bhandarkar Road': (18.5119, 73.8351),
        'Law College Road': (18.5164, 73.8347),
        'FC Road': (18.5291, 73.8418),
        'Vishrantwadi': (18.5860, 73.8789),
        'Satara Road': (18.4746, 73.8721),
        'Khadki': (18.5637, 73.8468),
        'Alandi Road': (18.6213, 73.8670),
        'Moshi': (18.6764, 73.8562)
    }
    return locations

# Initialize map without plotting all roads
def create_map(emergency_hospitals):
    map_pune = folium.Map(location=[emergency_hospitals['lat'].mean(), emergency_hospitals['lon'].mean()], zoom_start=12)

    # Adding hospitals to the map
    for idx, hospital in emergency_hospitals.iterrows():
        folium.Marker([hospital['lat'], hospital['lon']], 
                      popup=hospital['name'], 
                      icon=folium.Icon(color='blue', icon='info-sign')).add_to(map_pune)

    return map_pune

# Define heuristic function using Euclidean distance
def heuristic(node, goal):
    return geodesic((node[1], node[0]), (goal[1], goal[0])).meters

# A* algorithm using the road network (Manhattan distance)
def a_star(graph, start_node, goal_node):
    pq = []  # Priority queue
    heapq.heappush(pq, (0, start_node))  # (cost, node)
    came_from = {start_node: None}
    cost_so_far = {start_node: 0}
    
    while pq:
        current_cost, current_node = heapq.heappop(pq)
        
        if current_node == goal_node:
            break
        
        for neighbor in graph.neighbors(current_node):
            road_length = graph.edges[current_node, neighbor, 0]['length']
            new_cost = cost_so_far[current_node] + road_length
            
            if neighbor not in cost_so_far or new_cost < cost_so_far[neighbor]:
                cost_so_far[neighbor] = new_cost
                priority = new_cost + heuristic((graph.nodes[neighbor]['x'], graph.nodes[neighbor]['y']), 
                                                (graph.nodes[goal_node]['x'], graph.nodes[goal_node]['y']))
                heapq.heappush(pq, (priority, neighbor))
                came_from[neighbor] = current_node
                
    # Reconstruct path
    path = []
    current = goal_node
    while current != start_node:
        path.append(current)
        current = came_from[current]
    path.append(start_node)
    path.reverse()
    
    return path

# Function to add shortest path to the map
def plot_shortest_path(map_object, graph, path):
    route_coords = [(graph.nodes[node]['y'], graph.nodes[node]['x']) for node in path]
    folium.PolyLine(route_coords, color='red', weight=5, opacity=0.8).add_to(map_object)

# Main execution
locations = get_user_location()

# User selects a location
print("Select a location from the list:")
for idx, loc in enumerate(locations.keys()):
    print(f"{idx+1}. {loc}")

selected_idx = int(input("Enter the number corresponding to your location: ")) - 1
user_location_name = list(locations.keys())[selected_idx]
user_click_location = locations[user_location_name]
print(f"Selected location: {user_location_name} -> {user_click_location}")
# Load hospital data
hospital_data = pd.read_csv('pune_hospitals.csv')

# Filter for hospitals with emergency services
emergency_hospitals = hospital_data[hospital_data['emergency'] == 'yes'].copy()

# Convert 'geometry' column back to shapely objects
emergency_hospitals['geometry'] = emergency_hospitals['geometry'].apply(wkt.loads)

# Extract latitude and longitude from geometry
emergency_hospitals['lat'] = emergency_hospitals['geometry'].apply(lambda x: x.centroid.y if x.geom_type == 'Polygon' else x.y)
emergency_hospitals['lon'] = emergency_hospitals['geometry'].apply(lambda x: x.centroid.x if x.geom_type == 'Polygon' else x.x)


map_pune = create_map(emergency_hospitals)

# Find the nearest emergency hospital using Euclidean distance
emergency_hospitals['distance'] = emergency_hospitals.apply(
    lambda row: geodesic(user_click_location, (row['lat'], row['lon'])).meters, axis=1
)

# Get the nearest hospital
nearest_hospital = emergency_hospitals.loc[emergency_hospitals['distance'].idxmin()]
nearest_hospital_location = (nearest_hospital['lat'], nearest_hospital['lon'])

# Get the nearest nodes from the graph for both start and hospital
start_node = ox.distance.nearest_nodes(graph, user_click_location[1], user_click_location[0])
goal_node = ox.distance.nearest_nodes(graph, nearest_hospital_location[1], nearest_hospital_location[0])

# Run A* algorithm to find the shortest path
shortest_path = a_star(graph, start_node, goal_node)

# Plot the shortest path on the map
plot_shortest_path(map_pune, graph, shortest_path)

# Display the map with the shortest path
map_pune
